In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import httpx
import asyncio
import datetime as dt
import time
import pylint
import lxml
import fastparquet
import time

In [3]:
def round_date(date, scalar=1):
    while date.weekday() != 5:
        date = date + scalar*dt.timedelta(days=1)
    return date

In [4]:
def mod_generator(init_d, delta=1):
    init_d = round_date(init_d)
    curr = init_d
    while True:
        yield curr
        curr -= dt.timedelta(weeks=delta)

In [44]:
def catch_404(client):
    start_t = time.time()
    prev_t = start_t
    intervals = [260, 52, 26, 12, 4, 1]
    date = dt.datetime.strptime("1963-02-23", "%Y-%m-%d")
    for interval in intervals:
        gen = mod_generator(init_d=date,delta=interval)
        date = next(gen)
        while check_page(client.get(date.strftime('%Y-%m-%d')+"/")) == date:
            if (time.time() - prev_t) >=  30:
                prev_t = time.time()
                print(f"Current date of site: {date}")
            date = next(gen)
        print(f"the date {date} isn't equal to {check_page(client.get(date.strftime('%Y-%m-%d')+"/"))}, interval {interval} chaning at {time.time()-start_t} seconds")
        date += dt.timedelta(weeks=interval)

    return date

In [ ]:
def check_page(soup):
    str_d = soup.main.find('div', class_='pmc-paywall').p.string.split(maxsplit=2)[-1]
    return dt.datetime.strptime(str_d,'%B %d, %Y')
        


In [46]:
date = round_date(dt.datetime.strptime('1965-05-23','%Y-%m-%d'))
with httpx.Client(base_url="https://www.billboard.com/charts/hot-100/",timeout=10.0) as client:
    e = catch_404(client) 

the date 1958-03-01 00:00:00 isn't equal to 1958-08-04 00:00:00, interval 260 chaning at 0.5434670448303223 seconds
the date 1961-02-25 00:00:00 isn't equal to 1961-02-27 00:00:00, interval 52 chaning at 1.150543212890625 seconds
the date 1961-08-26 00:00:00 isn't equal to 1961-08-28 00:00:00, interval 26 chaning at 1.5880730152130127 seconds
the date 1961-12-02 00:00:00 isn't equal to 1961-12-04 00:00:00, interval 12 chaning at 2.025895118713379 seconds
the date 1961-12-30 00:00:00 isn't equal to 1962-01-06 00:00:00, interval 4 chaning at 2.5752062797546387 seconds
the date 1961-12-30 00:00:00 isn't equal to 1962-01-06 00:00:00, interval 1 chaning at 3.4651572704315186 seconds


In [33]:
e

datetime.datetime(1965, 5, 29, 0, 0)